In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
#import numpy as np
from numpy.linalg import norm
import psycopg2
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
#import multiprocessing

In [2]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

In [3]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

In [4]:
solution_dict ={"sudoku":os.path.join("projects","sudoku","solutions","ex01.json"),
                 "seeknumbers":os.path.join("projects","seeknumbers","solutions","ex01.json"),
                 "minotaur":os.path.join("projects","minotaur","solutions","level01.json"),
                 "creek":os.path.join("projects","creek","solutions","ex01.json"),
                 "yosenabe":os.path.join("projects","yosenabe","solutions","solution01.txt"),
                 "hop":os.path.join("projects","hop","solutions","solution01.txt"),
                 "lights":os.path.join("projects","lights","solutions","solution01.txt")}

In [5]:
def asp_try_5(asp_file,instance):
    models = []  
    errors = []
    symbols = []
    messages = []

    def custom_logger(code, message):
        messages.append(message)
        
    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10'], logger=custom_logger) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols, messages

In [6]:
def get_symbols_from_solution(solution_path):        
    with open(solution_path,'r') as file:
        if ".json" in solution_path:
            data = json.load(file)
            results = data["Call"][0]["Witnesses"] #[0]["Value"]
            #print(result)
        else:
            lines = file.readlines()
            splitted = lines[0].split()
            results = [{"Value":splitted}]
    return results

In [7]:
def jaccard_similarity(lista1, lista2):
    set1 = set(lista1)
    set2 = set(lista2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection/union

In [8]:
def get_metrics(symbols, solutions):
    symbols_set = set(symbols)
    matchs = 0
    final_tp = set()
    final_fp = ()
    final_fn = ()
    for solution in solutions:
        solution_set = set(solution["Value"])
        union = len(symbols_set.union(solution_set))
        tp = solution_set.intersection(symbols_set)
        matchs_temp = len(tp)
        solution_len = len(solution_set)
        if matchs_temp >= matchs:            
            matchs = matchs_temp
            final_tp = tp
            final_fp = symbols_set.difference(solution_set)
            final_fn = solution_set.difference(symbols_set)
            solution_len = len(solution_set)
    accuracy = matchs / solution_len if solution_len != 0 else 0 
    precision = matchs / (len(final_tp) + len(final_fp)) if (len(final_tp) + len(final_fp)) != 0 else 0  
    recall = matchs / (len(final_tp) + len(final_fn)) if (len(final_tp) + len(final_fn)) != 0 else 0 
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0
    jaccard = matchs / union if union != 0 else 0
    #print(f"Accuracy: {accuracy:.4f}. Precision: {precision:.4f}. Recall: {recall:.4f}. F1: {f1_score:.4f}")
    return matchs, round(accuracy,2), round(precision,2), round(recall,2), round(f1_score,2), round(jaccard,2)

In [9]:
def get_best_metrics(all_symbols,solutions):
    matchs = acc = prec = rec = f1 = jac = 0
    for symbols in all_symbols:
        matchs_t, acc_t, prec_t, rec_t, f1_t, jac_t = get_metrics(symbols, solutions)
        if matchs_t > matchs:
            matchs = matchs_t
            acc = acc_t
            prec = prec_t
            rec = rec_t
            f1 = f1_t
            jac = jac_t
    return matchs, acc, prec, rec, f1, jac

In [10]:
def repetition_counter(file):
    with open(file,'r') as f:
        lines = f.readlines()
        lines = [x for x in lines if x!='\n']
        #print(f"Lines {lines}")
        list_unique_lines = list(set(lines))
        unique_lines = len(list_unique_lines)
        #print(f"List Unique Lines: {list_unique_lines}")
        return len(lines) - unique_lines

In [11]:
def evaluate(file, instance, solution_path, canonical_file):
    models, errors, symbols, messages = asp_try_5(file,instance)
    n_errors = len(errors)
    errors = "".join(errors)
    n_models = len(models)    
    canonical_lines = lines_counter(canonical_file)
    encoding_lines = lines_counter(file)
    repeated_lines = repetition_counter(file)
    #print("Length of Canonical:" + str(canonical_lines) + ". Length Encoding: "+ str(encoding_lines))    
    diff_lines = abs(canonical_lines - encoding_lines)
    solutions = get_symbols_from_solution(solution_path)
    #print("Symbols: " + str(symbols))
    #print("Solution: " + str(solution))
    matchs, accuracy, precision, recall, f1_score, jaccard = get_best_metrics(symbols, solutions) 
    at_least_a_model = 1 if n_models > 0 else 0
    kpi = 25 + at_least_a_model*25 + 10*(accuracy + precision + recall + f1_score + jaccard) - 2*(diff_lines + repeated_lines + n_errors)
    result_dict = {"kpi":kpi, "diff_lines":diff_lines, "repeated":repeated_lines, "n_errors":n_errors, "n_models":n_models, "errors":errors, "symbols":symbols, "messages":messages, "matchs":matchs,
                    "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    #print(f"Precision: {precision}")
    print_out_dict = {x: result_dict[x] for x in result_dict if x not in ["symbols"]}
    print(print_out_dict)
    return result_dict

In [12]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [24]:
def save_in_database(conn, file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard):
    cursor = conn.cursor()
    symbols = str(symbols).replace("'","")
    messages = str(messages).replace("'","")
    command = '''INSERT INTO results (file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard) 
                VALUES ('%(file)s', '%(kpi)s', '%(diff_lines)s', '%(repeated)s', '%(n_errors)s', '%(n_models)s', '%(errors)s', '%(symbols)s', '%(messages)s', '%(matchs)s', '%(accuracy)s', '%(precision)s', '%(recall)s', '%(f1_score)s',
                 '%(jaccard)s');''' % {"file":file, "kpi": kpi, "diff_lines": diff_lines, "repeated":repeated, "n_errors": n_errors, "n_models": n_models, "errors":errors, "symbols": symbols, "messages":messages, "matchs":matchs, "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    #print(command)
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
        print(f"Insertion of results of file {file} successful")
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Result not inserted.")
        conn.commit()
        cursor.close()
        pass

In [25]:
def look_file(conn, file):
    cursor = conn.cursor()
    command = "SELECT * FROM results WHERE file = '"+ file + "'"
    results = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad search.")
        conn.commit()
        cursor.close()
        pass
    return results

## Evaluate Canonicals

In [26]:
#projects = [sudoku,seeknumbers,minotaur,creek,yosenabe,hop,lights]
#for project in projects:
#    story = project["story"]
#    print("======= Evaluation Canonical of " + story + " ===============")
#    canonical_file = canonical_dict[story]
#    instances_file = instances_dict[story]
#    solution_file = solution_dict[story]
#    results_dict  = evaluate(canonical_file,instances_file,solution_file,canonical_file)
    #kpi, n_models, errors, diff_lines, symbols, matchs, accuracy, precision, recall, f1_score, jaccard

## Evaluation from Database

In [27]:
def all_files_experiments(conn):
    cursor = conn.cursor()
    command = "SELECT file FROM experiments"
    results = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad search.")
        conn.commit()
        cursor.close()
        pass
    return results

In [28]:
experiments_files = [x[0] for x in all_files_experiments(conn)]

In [29]:
def all_files_results(conn):
    cursor = conn.cursor()
    command = "SELECT file FROM results"
    results = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad search.")
        conn.commit()
        cursor.close()
        pass
    return results

In [30]:
results_files = [x[0] for x in all_files_results(conn)]

In [31]:
selected_files = [ef for ef in experiments_files if ef not in results_files]

In [32]:
#selected_files.index('creek_from_yosenabe_and_hop_v3_20231203.lp')

In [33]:
tabu_files = ['lights_from_creek_and_seeknumbers_v2_20231203.lp','minotaur_from_sudoku_and_seeknumbers_v10_20231203.lp',
              'creek_from_yosenabe_and_hop_v3_20231203.lp','minotaur_from_seeknumbers_and_yosenabe_v3_20231203.lp',
              'minotaur_from_sudoku_and_lights_v3_20231203.lp','minotaur_from_hop_and_sudoku_v3_20231203.lp',
              'minotaur_from_seeknumbers_and_yosenabe_v4_20231203.lp','seeknumbers_from_minotaur_and_lights_v4_20231203.lp',
              'minotaur_from_sudoku_and_seeknumbers_v9_20231203.lp']

In [34]:
data = []
counter = 0
for file in selected_files:
    if file in tabu_files:
        continue
    counter += 1
    story = file.split("_")[0]
    results = look_file(conn,file)
    print("===Counter: "+ str(counter) +" ==== Evaluation of " + story + ". File: "+ file +" ===============")
    story = file.split("_")[0]
    canonical_file = canonical_dict[story]
    instance_file = instances_dict[story]
    solution_file = solution_dict[story]
    path_of_file = os.path.join("generated_solutions",file)
    #try:
    #if file != 'lights_from_creek_and_seeknumbers_v2_20231203.lp':
    #    continue
    results = evaluate(path_of_file,instance_file,solution_file,canonical_file)
    #continue
    kpi = results["kpi"]
    diff_lines = results["diff_lines"]
    repeated = results["repeated"]
    n_errors = results["n_errors"]
    n_models = results["n_models"]
    errors = results["errors"]
    symbols = results["symbols"]
    messages = results["messages"]
    matchs = results["matchs"]
    accuracy = results["accuracy"]
    precision = results["precision"]
    recall = results["recall"]
    f1_score = results["f1_score"]
    jaccard = results["jaccard"]
    save_in_database(conn, file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard)
    #except Exception as err:
    #    print(f"Unexpected {err=}, {type(err)=} Bad Evaluation.")
    #    #print(f"Problem with file {file}")
    #    pass
    data.append(results)

===Counter: 1 ==== Evaluation of creek. File: creek_from_minotaur_and_seeknumbers_v2_20231129.lp ===============
{'kpi': 17, 'diff_lines': 4, 'repeated': 0, 'n_errors': 0, 'n_models': 0, 'errors': '', 'messages': [], 'matchs': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'jaccard': 0}
Insertion of results of file creek_from_minotaur_and_seeknumbers_v2_20231129.lp successful
===Counter: 2 ==== Evaluation of creek. File: creek_from_minotaur_and_sudoku_v2_20231129.lp ===============
{'kpi': 9, 'diff_lines': 7, 'repeated': 0, 'n_errors': 1, 'n_models': 0, 'errors': 'parsing failed', 'messages': ['<block>:33:108-109: error: syntax error, unexpected =, expecting )\n'], 'matchs': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'jaccard': 0}
Insertion of results of file creek_from_minotaur_and_sudoku_v2_20231129.lp successful
===Counter: 3 ==== Evaluation of creek. File: creek_from_minotaur_and_yosenabe_v2_20231129.lp ===============
{'kpi': 13, 'diff_lines': 5, 

In [53]:
len(tabu_files)

9

## Insert Erronous Files

In [36]:
for file in tabu_files:
    story = file.split("_")[0]
    instance = instances_dict[story]
    solution_path = solution_dict[story]
    canonical_file = canonical_dict[story]
    models = errors = symbols = []
    n_errors = 1
    errors = "drastic timeout"
    messages = ["Drastic Timeout"]
    n_models = len(models)
    canonical_lines = lines_counter(canonical_file)
    encoding_lines = lines_counter(os.path.join("generated_solutions",file))
    repeated_lines = repetition_counter(os.path.join("generated_solutions",file))  
    diff_lines = abs(canonical_lines - encoding_lines)
    solutions = get_symbols_from_solution(solution_path)
    matchs = accuracy = precision = recall = f1_score = jaccard = 0
    at_least_a_model = 0
    kpi = 25 + at_least_a_model*25 + 10*(accuracy + precision + recall + f1_score + jaccard) - 2*(diff_lines + repeated_lines + n_errors)
    result_dict = {"kpi":kpi, "diff_lines":diff_lines, "repeated":repeated_lines, "n_errors":n_errors, "n_models":n_models, "errors":errors, "symbols":symbols, "messages":messages, "matchs":matchs,
                    "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    print_out_dict = {x: result_dict[x] for x in result_dict if x not in ["symbols"]}
    print(print_out_dict)
    save_in_database(conn, file, kpi, diff_lines, repeated_lines, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard)

{'kpi': -17, 'diff_lines': 20, 'repeated': 0, 'n_errors': 1, 'n_models': 0, 'errors': 'drastic timeout', 'messages': ['Drastic Timeout'], 'matchs': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'jaccard': 0}
INSERT INTO results (file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard) 
                VALUES ('lights_from_creek_and_seeknumbers_v2_20231203.lp', '-17', '20', '0', '1', '0', 'drastic timeout', '[]', '[Drastic Timeout]', '0', '0', '0', '0', '0',
                 '0');
Unexpected err=UniqueViolation('FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »unique_file_results«\nDETAIL:  Schlüssel »(file)=(lights_from_creek_and_seeknumbers_v2_20231203.lp)« existiert bereits.\n'), type(err)=<class 'psycopg2.errors.UniqueViolation'> Result not inserted.
{'kpi': -67, 'diff_lines': 45, 'repeated': 0, 'n_errors': 1, 'n_models': 0, 'errors': 'drastic timeout', 'messages': ['Drastic Ti

## Evaluation of a Version

In [33]:
version = 91

In [34]:
files = os.listdir("generated_solutions")
selected_files = [x for x in files if "v"+str(version) in x]
print(selected_files)

['creek_from_seeknumbers_and_lights_v91_20231116.lp', 'hop_from_seeknumbers_and_lights_v91_20231116.lp', 'lights_from_seeknumbers_and_hop_v91_20231116.lp', 'minotaur_from_seeknumbers_and_hop_v91_20231116.lp', 'seeknumbers_from_creek_and_hop_v91_20231116.lp', 'sudoku_from_yosenabe_and_lights_v91_20231116.lp', 'yosenabe_from_seeknumbers_and_sudoku_v91_20231116.lp']


In [35]:
for file in selected_files:
    story = file.split("_")[0]
    results = look_file(conn,file)
    if len(results) > 0:
        continue
    if file == 'minotaur_from_yosenabe_and_hop_v6_s1_20231031.lp':
        continue
    print("======= Evaluation of " + story + ". File: "+ file +" ===============")
    story = file.split("_")[0]
    canonical_file = canonical_dict[story]
    solution_file = solution_dict[story]
    #print(file)
    path_of_file = os.path.join("generated_solutions",file)
    result_dict = evaluate(path_of_file,instances_dict[story],solution_file,canonical_file)
    kpi = result_dict["kpi"]
    diff_lines = result_dict["diff_lines"]
    errors = result_dict["errors"]
    n_models, symbols, matchs, accuracy 
    n_models = len(models)
    save_in_database(conn, file, kpi, diff_lines, errors, n_models, symbols, matchs, accuracy)
    dict_element = {'story':story, 'file':file, 'kpi':kpi, 'diff_lines':diff_lines, 'errors':errors,  'version':version, 'symbols':symbols,'matchs':matchs}
    data.append(dict_element)

======= Evaluation of creek. File: creek_from_seeknumbers_and_lights_v91_20231116.lp ===============
KPI = 51. Diff_Lines: 24. Errors: 0. N_models: 1. Symbols: 15. Matchs: 0. Match_Ratio: 0
======= Evaluation of hop. File: hop_from_seeknumbers_and_lights_v91_20231116.lp ===============
KPI = 39. Diff_Lines: 36. Errors: 0. N_models: 1. Symbols: 15. Matchs: 0. Match_Ratio: 0
======= Evaluation of lights. File: lights_from_seeknumbers_and_hop_v91_20231116.lp ===============
KPI = 60. Diff_Lines: 15. Errors: 0. N_models: 1. Symbols: 23. Matchs: 0. Match_Ratio: 0
======= Evaluation of minotaur. File: minotaur_from_seeknumbers_and_hop_v91_20231116.lp ===============
KPI = 31.0. Diff_Lines: 49. Errors: 0. N_models: 1. Symbols: 36. Matchs: 1. Match_Ratio: 0.2
======= Evaluation of seeknumbers. File: seeknumbers_from_creek_and_hop_v91_20231116.lp ===============
Unexpected err=RuntimeError('parsing failed'), type(err)=<class 'RuntimeError'>.
KPI = -9. Diff_Lines: 34. Errors: 5. N_models: 0. Sym

## Evaluation of a all versions

In [13]:
versions2evaluate = [2, 3, 4, 9, 10]

In [ ]:
files = os.listdir("generated_solutions")
data = []
for version in versions2evaluate:
    selected_files = [x for x in files if "v"+str(version) in x in x]
    for file in selected_files:
        story = file.split("_")[0]
        results = look_file(conn,file)
        if len(results) > 0:
            continue
        if file == 'minotaur_from_yosenabe_and_hop_v6_s1_20231031.lp':
            continue
        print("======= Evaluation of " + story + ". File: "+ file +" ===============")
        story = file.split("_")[0]
        canonical_file = canonical_dict[story]
        solution_file = solution_dict[story]
        #print(file)
        path_of_file = os.path.join("generated_solutions",file)
        results = evaluate(path_of_file,instances_dict[story],solution_file,canonical_file)
        kpi = results["kpi"]
        diff_lines = results["diff_lines"]
        repeated = results["repeated"]
        errors = results["errors"]
        n_models = results["n_models"]
        symbols = results["symbols"]
        matchs = results["matchs"]
        accuracy = results["accuracy"]
        precision = results["precision"]
        recall = results["recall"]
        f1_score = results["f1_score"]
        jaccard = results["jaccard"]
        #save_in_database(conn, file, kpi, diff_lines, repeated, errors, n_models, symbols, matchs, accuracy, precision, recall, f1_score, jaccard)
        data.append(results)

In [17]:
df = pd.DataFrame.from_records(data)
df.sort_values(by='jaccard',ascending=False)

,kpi,diff_lines,errors,n_models,symbols,matchs,accuracy,precision,recall,f1_score,jaccard
28,91.00,9,0,1,"[black(1,1), black(1,2), black(1,3), black(1,4...",7,1.00,0.58,1.00,0.74,0.583333
86,62.75,31,0,1,"[sudoku(2,3,3), sudoku(3,3,8), sudoku(4,3,1), ...",61,0.75,0.50,0.75,0.60,0.429577
84,63.75,31,0,1,"[sudoku(2,3,3), sudoku(3,3,8), sudoku(4,3,1), ...",64,0.79,0.46,0.81,0.58,0.402516
34,86.00,14,0,1,"[path(1,2,1,1), path(2,1,1,1), path(1,1,1,2), ...",8,1.00,0.33,1.00,0.50,0.333333
85,75.00,25,0,1,"[cell(1,1), cell(2,1), cell(3,1), cell(4,1), c...",81,1.00,0.21,1.00,0.34,0.206633
...,...,...,...,...,...,...,...,...,...,...,...
27,50.00,0,0,0,[],0,0.00,0.00,0.00,0.00,0.000000
26,38.00,7,1,0,[],0,0.00,0.00,0.00,0.00,0.000000
25,43.00,2,1,0,[],0,0.00,0.00,0.00,0.00,0.000000
24,47.00,3,0,0,[],0,0.00,0.00,0.00,0.00,0.000000


In [12]:
df.groupby(['version'])['kpi'].mean()

version
1    26.25
2    20.75
3     2.25
4    27.50
5     8.75
Name: kpi, dtype: float64

In [14]:
df.groupby(['story'])['kpi'].mean()

story
creek          13.833333
minotaur       13.833333
seeknumbers    12.333333
sudoku         22.833333
Name: kpi, dtype: float64

In [26]:
file = "minotaur_from_sudoku_and_creek_v5_s1.lp"
with open(os.path.join('generated_solutions',file),'r') as toread:
    print(toread.read()) 

% Define the possible directions to move
direction(up).
direction(down).
direction(left).
direction(right).

% Define the possible movements of the Minotaur based on the current position
minotaur_move(X,Y,X+2,Y) :- minotaur(X,Y), not wall(X+1,Y), not wall(X+2,Y).
minotaur_move(X,Y,X-2,Y) :- minotaur(X,Y), not wall(X-1,Y), not wall(X-2,Y).
minotaur_move(X,Y,X,Y+2) :- minotaur(X,Y), not wall(X,Y+1), not wall(X,Y+2).
minotaur_move(X,Y,X,Y-2) :- minotaur(X,Y), not wall(X,Y-1), not wall(X,Y-2).

% Define the possible movements of the player based on the current position
player_move(X,Y,X+1,Y) :- player(X,Y), not wall(X+1,Y).
player_move(X,Y,X-1,Y) :- player(X,Y), not wall(X-1,Y).
player_move(X,Y,X,Y+1) :- player(X,Y), not wall(X,Y+1).
player_move(X,Y,X,Y-1) :- player(X,Y), not wall(X,Y-1).

% Define the goal state where the player is at the goal position
goal :- player(X,Y), goal(X,Y).

% Define the possible moves of the player and Minotaur
move :- player_move(X,Y,X1,Y1), minotaur_move(X2,Y

## Comparison of lines

In [4]:
projects = [sudoku,seeknumbers,minotaur,creek,yosenabe,hop,lights]

In [5]:
files = os.listdir("generated_solutions")

In [8]:
def lines_counter(file):
    with open(file,'r') as readfile:
        lines = readfile.readlines()
    return len(lines)

In [21]:
for project in projects:
    for version in range(6,9):
        selected_files = [x for x in files if "v"+str(version) in x and (story + "_from") in x]
        #print(selected_files)
        for file in selected_files:
            path_of_file = os.path.join("generated_solutions",file)
            story = project['story']
            print("Version: " + str(version) + ". Story: " + story + ". Lines: " + str(lines_counter(path_of_file)))

Version: 6. Story: sudoku. Lines: 24
Version: 6. Story: sudoku. Lines: 23
Version: 7. Story: sudoku. Lines: 11
Version: 8. Story: sudoku. Lines: 2
Version: 6. Story: seeknumbers. Lines: 19
Version: 6. Story: seeknumbers. Lines: 26
Version: 6. Story: seeknumbers. Lines: 14
Version: 7. Story: seeknumbers. Lines: 57
Version: 8. Story: seeknumbers. Lines: 14
Version: 6. Story: minotaur. Lines: 21
Version: 6. Story: minotaur. Lines: 32
Version: 6. Story: minotaur. Lines: 20
Version: 7. Story: minotaur. Lines: 13
Version: 8. Story: minotaur. Lines: 7
Version: 6. Story: creek. Lines: 31
Version: 6. Story: creek. Lines: 13
Version: 6. Story: creek. Lines: 16
Version: 7. Story: creek. Lines: 26
Version: 8. Story: creek. Lines: 23
Version: 6. Story: yosenabe. Lines: 23
Version: 6. Story: yosenabe. Lines: 42
Version: 6. Story: yosenabe. Lines: 23
Version: 7. Story: yosenabe. Lines: 17
Version: 8. Story: yosenabe. Lines: 8
Version: 6. Story: hop. Lines: 17
Version: 6. Story: hop. Lines: 17
Version

In [3]:
file = "minotaur_from_lights_and_creek_v2_20231201.lp"
with open(os.path.join('generated_solutions',file),'r') as toread:
    print(toread.read()) 

                    % Define all possible directions
dir(0,1). dir(0,-1). dir(1,0). dir(-1,0).
% Define the possible movements for the player
move(X,Y,DX,DY) :- at(X,Y,T), dir(DX,DY), not wall(X,Y,X+DX,Y+DY), T1 = T+1, at(X+DX,Y+DY,T1).
% Define the possible movements for the Minotaur
mino_move(X,Y,DX,DY) :- mino(X,Y,T), dir(DX,DY), not wall(X,Y,X+DX,Y+DY), T1 = T+1, mino(X+DX,Y+DY,T1).
% Define the possible movements for the Minotaur when the player is on the right or left
mino_move(X,Y,DX,0) :- mino(X,Y,T), at(PX,PY,T), DX = sign(PX-X), not wall(X,Y,X+DX,Y), T1 = T+1, mino(X+DX,Y,T1).
% Define the possible movements for the Minotaur when the player is up or down
mino_move(X,Y,0,DY) :- mino(X,Y,T), at(PX,PY,T), DY = sign(PY-Y), not wall(X,Y,X,Y+DY), T1 = T+1, mino(X,Y+DY,T1).
% Define the possible movements for the Minotaur when the player is in a diagonal
mino_move(X,Y,DX,DY) :- mino(X,Y,T), at(PX,PY,T), DX = sign(PX-X), DY = sign(PY-Y), not wall(X,Y,X+DX,Y+DY), T1 = T+1, mino(X+DX,Y